In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing=mp.solutions.drawing_utils #visualising poses
mp_pose=mp.solutions.pose #importing pose estimation model

In [2]:
def angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    
    rad=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    degree=np.abs(rad*180/np.pi)
    
    angle=min(degree,360-degree)
    return int(angle)

In [10]:
cap = cv2.VideoCapture(0)

#Training Monitoring
counter=0
stage=None
bool_swing=None

# Initialize Mediapipe Pose model
with mp.solutions.pose.Pose() as pose:
    while cap.isOpened():
        # Read frame from Webcam
        success, image = cap.read()
        if not success:
            break

        # Convert to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process image with Mediapipe Pose model
        results = pose.process(image)
        
        #Recolor to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Extract Landmarks
        try:
            landmarks=results.pose_landmarks.landmark
            
            #Compute angle
            left_wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_angle=angle(left_wrist,left_elbow,left_shoulder)
            
            left_hip=[landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            swing_angle=angle(left_hip,left_shoulder,left_elbow)
            
            #Output Angle
            cv2.putText(image, str(elbow_angle),tuple(np.multiply(left_elbow,[640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2,cv2.LINE_AA)
            
            #Counter Logic
            if elbow_angle>160:
                stage="down"
            if elbow_angle<50 and stage=="down":
                stage="up"
                counter+=1
            
            #Pose Correction Logic
            if swing_angle>25:
                bool_swing=1
            else:
                bool_swing=0

            
        except:
            pass
        
        #Display Counter
        cv2.rectangle(image,(0,0),(150,75),(0,0,0),-1)
        cv2.rectangle(image,(0,450),(640,480),(0,0,0),-1)
        cv2.putText(image,"REPS",(10,10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,str(counter),(15,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
        if bool_swing==1:
            cv2.putText(image,"Lock elbow in position",(15,465),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)


        # Draw pose landmarks
        if results.pose_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                                     mp_drawing.DrawingSpec(color=(225,105,65), thickness=2,circle_radius=2),
                                                     mp_drawing.DrawingSpec(color=(180,229,255), thickness=2,circle_radius=2))

        # Display webcam feed
        cv2.imshow('Feed',image)
        if cv2.waitKey(1) & 0xFF == ord('x'):
            break

    cap.release()
    cv2.destroyAllWindows()
